# Filtering keywords
Analysing the keywords in the desription and choosing the descriptive key words.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import csv

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.corpus import stopwords
stopWords = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to /Users/tjay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tjay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tjay/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
keywords = pd.read_csv('../../Data/overallCount.csv').sort_values(by=['count'], ascending=False)
keywords

,keyWord,count
6,and,347399
9,the,220975
26,a,179477
59,of,172927
38,with,120409
...,...,...
29626,pinotage-dominant,1
29623,blance,1
29620,sourish,1
29616,hickin,1


**Removing Stop Words.**

We can obeserve that the words with the highest count are not descriptive words (stopwords).\
These rows are removed in order to get descriptive words for better catogisation for the data set.

In [3]:
filteredKeywords = keywords.loc[(~keywords['keyWord'].isin(stopWords))]
filteredKeywords

,keyWord,count
27,wine,78038
58,flavors,62679
3,fruit,45010
0,aromas,39613
10,palate,38082
...,...,...
29626,pinotage-dominant,1
29623,blance,1
29620,sourish,1
29616,hickin,1


**Removing `wine` for obvious reasons.**

We also remove the word `wine` by logic as, well, we are analysing wine.

In [4]:
filteredKeywords1 = filteredKeywords.loc[(~filteredKeywords['keyWord'].isin(['wine']))]
filteredKeywords1

,keyWord,count
58,flavors,62679
3,fruit,45010
0,aromas,39613
10,palate,38082
21,acidity,34958
...,...,...
29626,pinotage-dominant,1
29623,blance,1
29620,sourish,1
29616,hickin,1


In [ ]:
# keep only nouns and adjectives

def wordTag(word):
    # returns word if keep word
    # returns false if not word of interest
    
    
    types = ['JJ', 'NN']
    try:
        res = nltk.pos_tag(word_tokenize(word.lower()))
        
        for word in res:
            if word[1] in types:
#                 returnable.append(word[0])
                return True
        
        
    except:
        pass
    
                           
    return False
    

In [32]:
selectedWords = {}

for index, row in filteredKeywords1.iterrows():
    if wordTag(row['keyWord']):
        if row['keyWord'] in selectedWords.keys():
            selectedWords[row['keyWord']] += np.float32(row['count'])
        else:
            selectedWords[row['keyWord']] = np.float32(row['count'])
            
filteredKeywords2 = pd.DataFrame.from_dict({'keyWord': [*selectedWords.keys()], 'count':[*selectedWords.values()]})
filteredKeywords2

,keyWord,count
0,fruit,45010.0
1,palate,38082.0
2,acidity,34958.0
3,finish,34943.0
4,drink,29967.0
...,...,...
34292,pinotage-dominant,1.0
34293,blance,1.0
34294,sourish,1.0
34295,hickin,1.0


**Keeping frequent keywords**

In order to derive useful catagories for machine learning models, we'll keep the top 25% (>7) keywords.

In [40]:
q3 = filteredKeywords2['count'].quantile(0.75)
selectedKeyWords = filteredKeywords2.loc[(filteredKeywords2['count'] > q3)]

In [39]:
with open("../../Data/topKeywords.csv", "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["keyWord", "count"])
        for index, row in filteredKeywords2.iterrows():
            writer
        outfile.close()

selectedKeyWords

,keyWord,count
0,fruit,45010.0
1,palate,38082.0
2,acidity,34958.0
3,finish,34943.0
4,drink,29967.0
...,...,...
8545,taransaud,6.0
8546,salt-and-pepper,6.0
8547,drappier,6.0
8548,patagonia,6.0
